In [1]:
import numpy as np
import random
import scipy.sparse as sp

In [2]:
from time import time
import pickle
import os

In [3]:
def E_score1(a,b):
    return np.sum(a * b) / (np.sqrt(
                    np.sum(np.power(a, 2))) * np.sqrt(np.sum(np.power(b, 2))) )

def E_score2(a,b):
    return np.sum(np.power(a-b, 2))

In [5]:
# Interaction-based Balanced Partition
def data_partition_1(train_items,k,T):

    with open('../data/ml-1m/user_pretrain.pk', 'rb') as f:
        uidW = pickle.load(f, encoding='latin1')
    with open('../data/ml-1m/item_pretrain.pk', 'rb') as f:
        iidW = pickle.load(f, encoding='latin1')

    # get_data_interactions_1
    data = []
    for i in train_items:
        for j in train_items[i]:
            data.append([i, j])

    # Randomly select k centroids
    max_data = 1.2 * len(data) / k
    centroids = random.sample(data, k)

    # centro emb
    centroembs = []
    for i in range(k):
        temp_u = uidW[centroids[i][0]]
        temp_i = iidW[centroids[i][1]]
        centroembs.append([temp_u, temp_i])

    for _ in range(T):
        C = [{} for i in range(k)]
        C_num=[0 for i in range(k)]
        Scores = {}
        for i in range(len(data)):
            for j in range(k):

                score_u = E_score2(uidW[data[i][0]],centroembs[j][0])
                score_i = E_score2(iidW[data[i][1]],centroembs[j][1])

                
                Scores[i, j] = -score_u * score_i


        Scores = sorted(Scores.items(), key=lambda x: x[1], reverse=True)

        fl = set()
        for i in range(len(Scores)):
            if Scores[i][0][0] not in fl:

                if C_num[Scores[i][0][1]] < max_data:
                    if data[Scores[i][0][0]][0] not in C[Scores[i][0][1]]:
                        C[Scores[i][0][1]][data[Scores[i][0][0]][0]]=[data[Scores[i][0][0]][1]]
                    else:
                        C[Scores[i][0][1]][data[Scores[i][0][0]][0]].append(data[Scores[i][0][0]][1])
                    fl.add(Scores[i][0][0])
                    C_num[Scores[i][0][1]] +=1

        centroembs_next = []
        for i in range(k):
            temp_u = []
            temp_i = []

            for j in C[i].keys():
                for l in C[i][j]:
                    temp_u.append(uidW[j])
                    temp_i.append(iidW[l])
            centroembs_next.append([np.mean(temp_u), np.mean(temp_i)])

        loss = 0.0

        for i in range(k):
            score_u = E_score2(centroembs_next[i][0],centroembs[i][0])

            score_i = E_score2(centroembs_next[i][1],centroembs[i][1])

            loss += (score_u * score_i)

        centroembs = centroembs_next
        print(_, loss)

    users = [[] for i in range(k)]
    items = [[] for i in range(k)]

    for i in range(k):
        users[i] = C[i].keys()
        for j in C[i].keys():
            for l in C[i][j]:
                if l not in items[i]:
                    items[i].append(l)


    return C,users,items

In [7]:
train_items = {}

with open('../data/ml-1m/all_interactions.txt') as f:
    for line in f.readlines():
        items = line.strip().split(" ")
        train_items[int(items[0])] = list(map(int, items[1:]))

In [8]:
C_p1, users_p1, items_p1 = data_partition_1(train_items, 10, 10)

0 0.006767361774109304
1 4.2092432504107287e-11
2 2.0469479984472325e-12
3 3.448909935384074e-14
4 3.158635474023927e-15
5 3.074886941238163e-16
6 6.783693527485079e-17
7 4.022157317624597e-17
8 3.355551089476239e-17
9 3.323299265488902e-17


In [3]:
users_p1[0][0]

NameError: name 'users_p1' is not defined

In [9]:
with open('../data/ml-1m/C_type-1_num-10.pk', 'rb') as f:
    interactions = pickle.load(f, encoding='latin1')

In [14]:
print("sagar here take this is partitioning type 1: interactions")
for i in interactions:
    print(sum([len(v) for k,v in i.items()]))

sagar here take this is partitioning type 1: interactions
120026
82291
120026
72271
75754
120026
100121
114374
120026
75294


In [15]:
with open('../data/ml-1m/C_type-2_num-10.pk', 'rb') as f:
    interactions = pickle.load(f, encoding='latin1')
    
print("sagar i am increasingly sad here take partition 2: users")
for i in interactions:
    print(sum([len(v) for k,v in i.items()]))

sagar i am increasingly sad here take partition 2: users
115338
119608
104089
116621
41104
41232
131828
110279
117305
102805


In [16]:
with open('../data/ml-1m/C_type-3_num-10.pk', 'rb') as f:
    interactions = pickle.load(f, encoding='latin1')
    
print("nyeh. partition 3: random")
for i in interactions:
    print(sum([len(v) for k,v in i.items()]))

nyeh. partition 3: random
100020
100020
100020
100020
100020
100020
100020
100020
100020
100029


In [36]:
# User-based Balanced Partition
def data_partition_2(train_items,k,T):
    data = train_items

    with open('../data/ml-1m/item_pretrain.pk', 'rb') as f:
        #load encodings
        uidW = pickle.load(f, encoding='latin1')

    # Randomly select k centroids
    max_data = 1.2 * len(data) / k
    centroids = random.sample(data.keys(), k)

    # centroids with embeddings
    centroembs = []
    for i in range(k):
        centroembs.append(uidW[centroids[i]])

    # what
    for _ in range(T):
        C = [{} for i in range(k)]
        Scores = {}
        for i in data.keys():
            for j in range(k):
                # calculate some score between item and centroid embeddings
                score_u = E_score2(uidW[i],centroembs[j])
                # Store (item, centroid) pairings for all pairs
                Scores[i, j] = -score_u
        
        # sort scores lowest to highest
        Scores = sorted(Scores.items(), key=lambda x: x[1], reverse=True)
        
        fl = set()
        # length of scores should = num items
        for i in range(len(Scores)):
            # Scores[i][0][0] = movie index, Scores[i][0][1] = cluster index
            # if an item has not been sorted yet
            if Scores[i][0][0] not in fl:
                # Scores[i][0][1] contains the cluster for that item cluster pair
                if len(C[Scores[i][0][1]]) < max_data:
                    # this is typically mapping (user, cluster) = movies but we don't need that
                    # with the new data, we're goint to have (movie, cluster) = users
                    C[Scores[i][0][1]][Scores[i][0][0]] = data[Scores[i][0][0]]
                    # mark item as clustered
                    fl.add(Scores[i][0][0])

        # make new centroids
        centroembs_next = []
        for i in range(k):
            temp_u = []
            for u in C[i].keys():
                temp_u.append(uidW[u])
            centroembs_next.append(np.mean(temp_u))

        # calculate loss
        loss = 0.0
        for i in range(k):
            score_u = E_score2(centroembs_next[i],centroembs[i])
            loss += score_u

        centroembs = centroembs_next
        print(_, loss)

    # how do we then create the correct partitioning format?
    # rn, data = movie:users
    
    # create partitions
    users = [[] for i in range(k)]
    items = [[] for i in range(k)]
    
    # Currently, C contains (movie, cluster) = users
    # iterate through num clusters
    for i in range(k):
        items[i]=C[i].keys()
#         users[i]=C[i].keys()
        for j in C[i].keys():
            for l in C[i][j]:
                if l not in users[i]:
                    users[i].append(l)

    return C,users,items

In [14]:
train_items = {}

with open('../data/ml-1m/all_interactions.txt') as f:
    for line in f.readlines():
        items = line.strip().split(" ")
        train_items[int(items[0])] = list(map(int, items[1:]))

In [31]:
train_items_swap = {}

for k,v in train_items.items():
    for movie in v:
        if movie in train_items_swap:
            train_items_swap[movie].append(k)
        else:
            train_items_swap[movie] = [k]

In [18]:
movies = set()
for movielist in train_items.values():
    movies.update(movielist)
movies = list(movies)

In [37]:
C_p2, users_p2, items_p2 = data_partition_2(train_items_swap, 10, 10)

0 0.36463903449475765
1 3.148407227011194e-05
2 1.0472792529069178e-05
3 3.957271205673246e-06
4 1.6367231637158654e-07
5 7.015341229482164e-08
6 3.6588507876895646e-08
7 2.706808854662635e-08
8 2.453024589011226e-08
9 2.3489976732025487e-08


In [46]:
items_p2_1 = [list(i) for i in items_p2]

In [48]:
with open('../data/ml-1m/C_I_type-4_num-10.pk', 'wb') as handle:
    pickle.dump(items_p2_1, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
import pandas as pd

users_partition = pd.read_pickle('../data/ml-1m/old_encodings/C_U_type-2_num-10.pk')

In [88]:
df = pd.read_csv('../data/ml-1m/old_encodings/C_U_type-2_num-10.csv')
df

,UserID,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
0,0,0,0,0,0,1,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,0
2,2,0,0,0,0,0,0,1,0,0,0
3,3,0,0,0,0,0,0,0,1,0,0
4,4,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6035,6035,0,0,0,0,1,0,0,0,0,0
6036,6036,0,0,0,0,1,0,0,0,0,0
6037,6037,0,1,0,0,0,0,0,0,0,0
6038,6038,0,0,0,0,1,0,0,0,0,0


In [24]:
#Random
def data_partition_3(train_items,k):

    data = []
    for i in train_items:
        for j in train_items[i]:
            data.append([i, j])

    index = list(range(len(data)))
    random.shuffle(index) 
    elem_num = len(data) // k
    C = [{} for i in range(k)]

    for idx in range(k):
        start = idx*elem_num
        if idx!= k-1:
            end = (idx+1)*elem_num
        else:
            end = len(data)
        for i in index[start:end]:
            if data[i][0] not in C[idx]:
                C[idx][data[i][0]]=[data[i][1]]
            else:
                C[idx][data[i][0]].append(data[i][1])

    users = [[] for i in range(k)]
    items = [[] for i in range(k)]
    for i in range(k):
        users[i]=C[i].keys()
        for j in C[i].keys():
            for l in C[i][j]:
                if l not in items[i]:
                    items[i].append(l)

    return C,users,items

In [25]:
C_p3, users_p3, items_p3 = data_partition_3(train_items, 10)

In [55]:
users_p3_l = [list(i) for i in users_p3]

truthfully I have no idea what the formatting of "C" is, so I just wrote it to a file as is, and we'll see what's up later ig